In [1]:
import os 
import pandas as pd
import json
from pathlib import Path
from sqlalchemy import create_engine, text, extract
from sqlalchemy.ext.automap import automap_base


In [2]:
gamedata = pd.read_csv("archive\games.csv", index_col=False)
revenuedata = pd.read_csv("archive\Steam_2024_bestRevenue_1500.csv", index_col=False)

In [3]:
dfgames = pd.DataFrame(gamedata)

dfrevenue = pd.DataFrame(revenuedata)
dfrevenue.reset_index(drop=True,inplace=True)

In [4]:
# NOT NULL CONSTRAINTS FOR GENERAL INFO TABLE
dfgames["Name"].fillna("No Name?", inplace=True)
dfgames["Tags"].fillna("No Tag", inplace=True)
dfgames["Genres"].fillna("No Genres", inplace=True)
dfgames["Categories"].fillna("No Categories", inplace=True)
dfgames["Publishers"].fillna("No Publisher", inplace=True)
dfgames["Developers"].fillna("No Developer", inplace=True)
dfgames.head()

# NOT NULL CONSTRAINTS FOR REVENUE TABLE
dfrevenue["developers"].fillna("No Developer", inplace=True)
dfrevenue["publishers"].fillna("No Publisher", inplace=True)
dfrevenue["steamId"].fillna("No steamId", inplace=True)
dfrevenue["avgPlaytime"].fillna("No one plays this game", inplace=True)
dfrevenue["publisherClass"].fillna("No Class", inplace=True)
dfrevenue["reviewScore"].fillna("No Score", inplace=True)
dfclean = dfgames.drop(columns=[ 'Discount','DLC count', 'About the game','Supported languages', 'Full audio languages',
                           'Reviews','Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux','Metacritic url', 'Positive', 'Negative','Score rank','Achievements', 'Recommendations','Notes','Screenshots', 'Movies'])



In [5]:
dfclean.to_csv("archive\cleangames.csv")
dfrevenue.to_csv("archive\cleanrevenue.csv")

In [6]:
database_path = "sqlite:///archive/revenue.sqlite"
engine = create_engine(database_path)
conn= engine.connect()

In [8]:
free_vs_paid_query="""SELECT
    CASE 
        WHEN price = 0 THEN 'Free'
        ELSE 'Paid'
    END AS price_category,
    AVG(revenue) AS avg_revenue,
    AVG(CAST(copies_sold AS INTEGER)) AS avg_copies_sold,
    AVG(avg_playtime) AS avg_playtime,
    AVG(review_score) AS avg_review_score
FROM revenue_data
GROUP BY price_category"""

free_paid_df = pd.read_sql_query(free_vs_paid_query, conn)
free_paid_df

,price_category,avg_revenue,avg_copies_sold,avg_playtime,avg_review_score
0,Free,2.820023e+06,687998.682353,45.469488,63.329412
1,Paid,2.621110e+06,108652.980212,10.585971,76.974558


In [9]:
revenue_trend= """
SELECT 
    CASE STRFTIME('%m', formatted_date)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS release_month,
    AVG(revenue) AS avg_monthly_revenue
FROM (
    SELECT 
        revenue, 
        DATE(SUBSTR(release_date, 7, 4) || '-' || 
             SUBSTR(release_date, 4, 2) || '-' || 
             SUBSTR(release_date, 1, 2)) AS formatted_date
    FROM revenue_data
    WHERE release_date IS NOT NULL
)
GROUP BY STRFTIME('%m', formatted_date)
ORDER BY STRFTIME('%m', formatted_date);
"""

rev_trend_df = pd.read_sql_query(revenue_trend, conn)
rev_trend_df

,release_month,avg_monthly_revenue
0,January,4.437570e+06
1,February,5.313502e+06
2,March,1.314599e+06
3,April,1.422114e+06
4,May,1.465438e+06
5,June,1.547320e+06
6,July,1.101356e+06
7,August,5.916519e+06
8,September,9.471220e+05


In [11]:
playtime="""SELECT *
FROM (
    SELECT 
        CASE
            WHEN avg_playtime < 5 THEN 'Under 5 hrs'
            WHEN avg_playtime BETWEEN 5 AND 10 THEN '5–10 hrs'
            WHEN avg_playtime BETWEEN 10 AND 20 THEN '10–20 hrs'
            WHEN avg_playtime BETWEEN 20 AND 50 THEN '20–50 hrs'
            WHEN avg_playtime BETWEEN 50 AND 100 THEN '50–100 hrs'
            ELSE '100+ hrs'
        END AS playtime_bucket,
        COUNT(*) AS game_count,
        AVG(revenue) AS avg_revenue,
        AVG(review_score) AS avg_review_score
    FROM revenue_data
    GROUP BY playtime_bucket
)
ORDER BY 
    CASE playtime_bucket
        WHEN 'Under 5 hrs' THEN 1
        WHEN '5–10 hrs' THEN 2
        WHEN '10–20 hrs' THEN 3
        WHEN '20–50 hrs' THEN 4
        WHEN '50–100 hrs' THEN 5
        WHEN '100+ hrs' THEN 6
    END"""

playtime_df = pd.read_sql_query(playtime, conn)
playtime_df

,playtime_bucket,game_count,avg_revenue,avg_review_score
0,Under 5 hrs,564,2.192043e+05,76.867021
1,5–10 hrs,419,5.440303e+05,76.451074
2,10–20 hrs,284,2.123665e+06,75.869718
3,20–50 hrs,188,1.405096e+07,74.904255
4,50–100 hrs,32,9.858947e+06,76.687500
5,100+ hrs,13,2.831132e+06,64.076923


In [10]:
publisher_ave_rev= """SELECT publishers, AVG(revenue) AS avg_revenue, COUNT(*) AS game_count FROM revenue_data GROUP BY publishers ORDER BY avg_revenue DESC"""

pub_rev_df = pd.read_sql_query(publisher_ave_rev, conn)
pub_rev_df

,publishers,avg_revenue,game_count
0,Game Science,8.377934e+08,1
1,Pocketpair,3.923286e+08,1
2,Newnight,2.170179e+08,1
3,PlayStation Publishing LLC,1.272213e+08,4
4,NEXON,1.022448e+08,1
...,...,...,...
1127,Y Press Games,2.102200e+04,1
1128,SRG Studios,2.095500e+04,1
1129,Pizza Bear Games,2.094600e+04,1
1130,IzanagiGames,2.092200e+04,1
